<a href="https://colab.research.google.com/github/beyza720/CENG463-Assignment2/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch
!pip install evaluate

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

from google.colab import drive
drive.mount('/content/drive')

train_path = '/content/drive/My Drive/Colab Notebooks/orientation-lv-train.tsv'
train_orientation_data = pd.read_csv(train_path, sep='\t')

class_0 = train_orientation_data[train_orientation_data['label'] == 0]
class_1 = train_orientation_data[train_orientation_data['label'] == 1]

class_0_oversampled = resample(class_0, replace=True, n_samples=len(class_1), random_state=42)
balanced_train_orientation_data = pd.concat([class_1, class_0_oversampled])

train_orientation, val_orientation = train_test_split(
    balanced_train_orientation_data,
    test_size=0.1,
    stratify=balanced_train_orientation_data['label'],
    random_state=42
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-multilingual-cased", num_labels=2
)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_orientation)
val_dataset = Dataset.from_pandas(val_orientation)

def tokenize_function(examples):
    return tokenizer(examples["text_en"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1130 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    report_to="none",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=71,
    save_steps=500,
    save_total_limit=2
)

In [12]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [13]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [14]:
from transformers import Trainer

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.566600,0.416666,0.849206,0.849294,0.849206,0.849197
2,0.328400,0.231111,0.944444,0.944556,0.944444,0.944441
3,0.206800,0.230944,0.920635,0.922337,0.920635,0.920555


TrainOutput(global_step=213, training_loss=0.36727949599145165, metrics={'train_runtime': 76.9443, 'train_samples_per_second': 44.058, 'train_steps_per_second': 2.768, 'total_flos': 891946477670400.0, 'train_loss': 0.36727949599145165, 'epoch': 3.0})

In [18]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
train_encodings = tokenizer(list(train_orientation['text']), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_orientation['text']), truncation=True, padding=True, max_length=512)

In [20]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = Dataset(train_encodings, train_orientation['label'].tolist())
val_dataset = Dataset(val_encodings, val_orientation['label'].tolist())

In [21]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",  # save_strategy, evaluation_strategy ile aynı olmalı
    report_to="none",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

<ipython-input-22-f2a888a7213a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.669400,0.604479
2,0.458100,0.294116
3,0.294800,0.209024


TrainOutput(global_step=213, training_loss=0.5066423981402401, metrics={'train_runtime': 108.4084, 'train_samples_per_second': 31.271, 'train_steps_per_second': 1.965, 'total_flos': 891946477670400.0, 'train_loss': 0.5066423981402401, 'epoch': 3.0})

In [25]:
val_preds = trainer.predict(val_dataset)
predicted_labels = torch.argmax(torch.tensor(val_preds.predictions), axis=1).numpy()
true_labels = val_orientation['label'].tolist()

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

import pandas as pd
performance_table = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1 Score"],
    "Value": [accuracy, precision, recall, f1]
})

print("\nModel: XLM-RoBERTa Base")
print(performance_table.to_string(index=False))

print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))


Model: XLM-RoBERTa Base
   Metric    Value
 Accuracy 0.928571
Precision 0.929545
   Recall 0.928571
 F1 Score 0.928531

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93        63
           1       0.95      0.90      0.93        63

    accuracy                           0.93       126
   macro avg       0.93      0.93      0.93       126
weighted avg       0.93      0.93      0.93       126

